## Tensorflow の学習済みモデルを学習に利用する
### 学習済みモデルのダウンロード
学習済みモデルがS3にアップロードされていれば、学習インスタンスで読み込んで、学習済みモデルから学習をスタートすることできます。
まずは、tensorflowのresnet_v1_50のモデルをダウンロード・解凍します。

In [184]:
import urllib.request
import os

download_file = "resnet_v2_152_2017_04_14.tar.gz"
if not os.path.exists(download_file):
    url = 'http://download.tensorflow.org/models/resnet_v2_152_2017_04_14.tar.gz'
    urllib.request.urlretrieve(url, download_file)
    
!tar xvzf $download_file

inception_resnet_v2_2016_08_30.ckpt


## データのダウンロード

### Validation dataのGround Truthのダウンロード

Tensorflowの学習済みモデルが出力するラベルIDは、ILSVR2012のサイトで確認できるものとは異なり、caffeモデルで採用されているラベルIDとなります、従って、Validation dataのGround TruthのIDを、caffeモデルを配布しているサイト(http://dl.caffe.berkeleyvision.org/caffe_ilsvrc12.tar.gz) からダウンロードします。　Validation dataのGround Truthは、`./gt/val.txt`におかれます。

In [195]:
!wget http://dl.caffe.berkeleyvision.org/caffe_ilsvrc12.tar.gz
!mkdir gt
!tar -xf caffe_ilsvrc12.tar.gz -C ./gt

--2019-01-13 02:00:48--  http://dl.caffe.berkeleyvision.org/caffe_ilsvrc12.tar.gz
Resolving dl.caffe.berkeleyvision.org (dl.caffe.berkeleyvision.org)... 169.229.222.251
Connecting to dl.caffe.berkeleyvision.org (dl.caffe.berkeleyvision.org)|169.229.222.251|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17858008 (17M) [application/octet-stream]
Saving to: ‘caffe_ilsvrc12.tar.gz.1’

caffe_ilsvrc12.tar. 100%[===================>]  17.03M  18.0MB/s    in 0.9s    

2019-01-13 02:00:49 (18.0 MB/s) - ‘caffe_ilsvrc12.tar.gz.1’ saved [17858008/17858008]



### Validation dataの画像データのダウンロード

画像データはILSVRC 2012で配布されているもの(http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_val.tar) を利用します。非常に大きく、ダウンロードに数時間かかるため、適当なところで停止(■を押すなど)してください。ダウンロードできたところまで展開して、`./images`に保存されます。

In [13]:
!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_val.tar
!tar -xf ILSVRC2012_img_val.tar
!mkdir images
!mv *.JPEG ./images

--2019-01-12 11:58:21--  http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_val.tar
Resolving www.image-net.org (www.image-net.org)... 171.64.68.16
Connecting to www.image-net.org (www.image-net.org)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘ILSVRC2012_img_val.tar’

ILSVRC2012_img_val.   0%[                    ]  22.99M  1.47MB/s    eta 70m 3s ^C
tar: Unexpected EOF in archive
tar: rmtlseek not stopped at a record boundary
tar: Error is not recoverable: exiting now


## モデルのロード
モデルのロードの手順は以下のとおりです。
1. 入力のテンソルを定義します。画像サイズや値の標準化を行います。
2. `nets.resnet_v2.resnet_v2_152`を利用してモデルを構築します。
3. ckptファイルを読み込んで、学習済みのパラメータで初期化します。

In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np

import tensorflow.contrib.slim as slim
from  tensorflow.contrib.slim import nets

tf.reset_default_graph()

checkpoint_file = 'resnet_v2_152.ckpt'

input_tensor = tf.placeholder(tf.float32, shape=(None,299,299,3), name='input_image')
scaled_input_tensor = tf.scalar_mul((1.0/255), input_tensor)
scaled_input_tensor = tf.subtract(scaled_input_tensor, 0.5)
scaled_input_tensor = tf.multiply(scaled_input_tensor, 2.0)

#Load the model
sess = tf.Session()
with slim.arg_scope(nets.resnet_v2.resnet_arg_scope()):
    logits,end_points = nets.resnet_v2.resnet_v2_152(inputs=scaled_input_tensor, num_classes=1001, is_training=False)
saver = tf.train.Saver()
saver.restore(sess, checkpoint_file)

INFO:tensorflow:Restoring parameters from resnet_v2_152.ckpt


画像を読み込んでテストしてみます。表示されるラベルは、ダウンロードしたGround Truthのラベルよりも1多いことに注意してください（Resnet v2では背景IDが最初に追加されているためです）。

In [3]:
im = Image.open('images/ILSVRC2012_val_00001545.JPEG').resize((299,299))
im = np.array(im)
im = im.reshape(-1,299,299,3)
prob = sess.run(end_points['predictions'], feed_dict={input_tensor: im})

prob = prob.reshape(-1)

# top-5 labels (but not sorted)
ind = np.argpartition(prob, -5)[-5:]

# top-5 labels
# Taking negtive (-) yields descending order
sorted_class = ind[np.argsort(-prob[ind])]
print(sorted_class - 1) # The first class_id is background, which should be removed
print(prob[sorted_class])

[550 505 967 827 899]
[7.4968690e-01 2.4663955e-01 9.3139079e-04 1.4105994e-04 7.6621385e-05]


## Tensorflow serving 用のモデルにexport

### S3への学習済みモデルのアップロード
SageMaker Python SDKを利用して、ファイル resnet_v1_50.ckpt をS3にアップロードします。この学習済みモデルはtf.slimのモデルなので、Tensorflowのスクリプト(cifar100.py)ではtf.slimのモデルとして読み込まれるようにします。

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

bucket_name = sagemaker_session.default_bucket()
prefix_name  = 'pretrained_model/resnet_tf'
file_name = 'resnet_v1_50.ckpt'
model_file = sagemaker_session.upload_data(path=file_name, bucket=bucket_name, key_prefix=prefix_name)
print('Your pretrained model is uploaded to: {}'.format(model_file))

### 学習データのアップロード
- 今回はtf.kerasのデータセットからcifar100をダウンロードして利用します。
- `convert_to`の関数でtfrecord形式に変換して、アップロードします

In [ ]:
import os
import tensorflow as tf

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def convert_to(data_set, filename):
    """Converts a dataset to tfrecords."""
    images = data_set["images"]
    labels = data_set["labels"]
    num_examples = images.shape[0]

    rows = images.shape[1]
    cols = images.shape[2]
    if len(images.shape) > 3:
        depth = images.shape[3]
    else:
        depth = 1
        
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(int(labels[index])),
            'image_raw': _bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
train = {"images": x_train, "labels": y_train}
test= {"images": x_test, "labels": y_test}
os.makedirs("./data", exist_ok=True)
convert_to(train, "./data/train.tfrecords")
convert_to(test, "./data/test.tfrecords")
input_data = sagemaker_session.upload_data(path='./data', key_prefix='pretrained_model/data')
print('Your pretrained model is uploaded to: {}'.format(input_data))

## SageMakerでの学習

### Local mode用スクリプトダウンロード

デバッグでLocal Modeを実行するために、スクリプトをダウンロードして実行します。

In [ ]:
download_file = "setup.sh"
if not os.path.exists(download_file):
    url = 'https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/sagemaker-python-sdk/tensorflow_distributed_mnist/setup.sh'
    urllib.request.urlretrieve(url, download_file)
    
!sh setup.sh

### 学習済みモデルに対する転移学習の実行
- S3にある学習済みモデルの場所をbucket_nameとprefix_nameで渡します。
- S3の学習データの場所はinput_dataとしてfitに渡します。
- `cifar100.py`内での読み込みは以下のとおりです。boto3でファイルをダウンロード、init_from_checkpointで読み込みます。 

(参考)
- https://github.com/tensorflow/tensorflow/issues/14713
- https://stackoverflow.com/questions/47867748/transfer-learning-with-tf-estimator-estimator-framework

```python
    bucket_name = params['bucket_name']
    prefix_name = params['prefix_name']
    s3 = boto3.resource('s3')
    try:
        s3.Bucket(bucket_name).download_file(prefix_name, 'resnet.ckpt')
        print("Pretrained model is downloaded.")
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
            
            ...
    tf.train.init_from_checkpoint("./resnet.ckpt",{v.name.split(':')[0]: v for v in variables_to_restore if not 'biases' in v.name})
```

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='cifar100.py',
                             role=role,
                             framework_version='1.11.0',
                             training_steps=100, 
                             evaluation_steps=10,
                             train_instance_count=1,
                             train_instance_type='local',
                             hyperparameters={'bucket_name': bucket_name,
                                                                 'prefix_name': prefix_name +"/" + file_name})

estimator.fit(input_data)